# Creating Azure Resources



In [132]:
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials
import json

# Set resources' parameters
SUBSCRIPTION_NAME = 'Team Danielle Internal'
SUBSCRIPTION_ID = 'edf507a2-6235-46c5-b560-fd463ba2e771'
SP_CLIENT_ID = '095d4aa7-a8d5-4f24-aa96-c631e3dee97c'
SP_SECRET = 'a9b478dc-dbdb-403f-bd6d-1dee6c0b1bd4'
SP_TENANT = '72f988bf-86f1-41af-91ab-2d7cd011db47'

RESOURCE_GROUP = 'baimm'
LOCATION = 'eastus'
ACR_NAME = 'baimmacr'
BAI_DOCKER_IMG = 'baimmacr.azurecr.io/baimmimg:v1'
SCHED_DOCKER_IMG = 'baimmacr.azurecr.io/baimmschedimg:v1'

BAI_CLUSTER_NAME = 'baimmcluster'
BAI_WORKSPACE = 'baimmws'
BAI_USER = 'baimmuser'
BAI_PASS = 'baimmpass'
BAI_VM_SIZE = 'Standard_D2'
BAI_VM_IMG = 'UbuntuLTS'
BAI_NODES_MIN = 0
BAI_NODES_MAX = 3
BLOB_ACCOUNT = 'baimmstorage'
MODELS_CONTAINER = 'models'
PREDS_CONTAINER = 'preds'
DATA_CONTAINER = 'data'
DATA_FILE = 'sensor_data.csv'


In [ ]:
# Login to subscription
!az login

In [ ]:
# Select subscription
!az account set -s "{SUBSCRIPTION_NAME}"

In [ ]:
# Create resource group
!az group create -l {LOCATION} -n {RESOURCE_GROUP}

In [ ]:
# Create ACR
!az acr create --resource-group {RESOURCE_GROUP} --name {ACR_NAME} --sku Basic

In [ ]:
# Create Blob storage account
!az storage account create -n {BLOB_ACCOUNT} -g {RESOURCE_GROUP} -l {LOCATION}

In [ ]:
# Retrieve Blob storage key
blob_keys = !az storage account keys list -g {RESOURCE_GROUP} -n {BLOB_ACCOUNT} --output tsv
blob_key = blob_keys[0].split('\t')[2]

In [ ]:
# Create models, predictions and data containers
!az storage container create -n {MODELS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {PREDS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {DATA_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}

In [ ]:
# Create Batch AI cluster and set auto-scaling
!az batchai workspace create -g {RESOURCE_GROUP} -n {BAI_WORKSPACE}
!az batchai cluster create -g {RESOURCE_GROUP} -n {BAI_CLUSTER_NAME} -w {BAI_WORKSPACE} -s {BAI_VM_SIZE} -i {BAI_VM_IMG} --min {BAI_NODES_MIN} --max {BAI_NODES_MAX} -u {BAI_USER} -p {BAI_PASS} --storage-account-name {BLOB_ACCOUNT} --storage-account-key {blob_key} --bfs-name {BLOB_CONTAINER}
!az batchai cluster auto-scale -g {RESOURCE_GROUP} -w {BAI_WORKSPACE} -n {BAI_CLUSTER_NAME} --min {BAI_NODES_MIN} --max {BAI_NODES_MAX}

In [136]:
# Create config file for scoring script
score_config = {"blob_account": BLOB_ACCOUNT,
                "blob_key": blob_key,
                "models_blob_container": MODELS_CONTAINER,
                "predictions_blob_container": PREDS_CONTAINER}

with open('batchai/predict_config.json', 'w') as f:
    json.dump(score_config, f, indent=4)


In [ ]:
# Create Batch AI docker img
!docker build -f batchai/Dockerfile -t {BAI_DOCKER_IMG} .

In [ ]:
# Validate that the image was created
!docker images

In [ ]:
# Login to ACR and push docker image
!az acr login --name {ACR_NAME}
!docker push {BAI_DOCKER_IMG}

In [ ]:
# Copy models from local dir to blob container
!az storage blob upload-batch -d {MODELS_CONTAINER} -s models --account-name {BLOB_ACCOUNT}

In [ ]:
# Copy dataset to blob
!az storage blob upload -c {DATA_CONTAINER} -f data/{DATA_FILE} -n {DATA_FILE} --account-name {BLOB_ACCOUNT}

In [ ]:
!ls